In [1]:
import os
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

import matplotlib
matplotlib.style.use('des_dr1')

In [2]:
12*1024**2

12582912

In [3]:
summed = 0
for ell in np.arange(3072):
    summed += (2*ell)+1

In [8]:
np.sqrt(4*np.pi/(12*1024**2))

np.float64(0.0009993424882289927)

In [11]:
np.pi/3072

0.0010226538585904274

In [2]:
nside = 1024
nside_ls = 3*nside

In [3]:
boone_dir = '/n/netscratch/dvorkin_lab/Lab/kboone/cmb_conv/'
fid_dir = boone_dir + 'noiseless_cls/'

all_files = np.array(os.listdir(fid_dir))
fid_inds = [i for i, s in enumerate(all_files) if s.startswith('f')]
fid_files = all_files[fid_inds]

Cls = np.zeros((len(fid_files), 3072), dtype = np.float64)
for i in np.arange(len(fid_files)):
    Cls[i] = np.load(fid_dir + fid_files[i])
ave_sig = np.average(Cls, axis = 0)

In [4]:
ls = np.arange(nside_ls)
ls_ACT = np.arange(2091)

In [5]:
N = 1e7
x = [34.19233078337337	,71.86940802505276	,127.50974815233886	,208.08224631676606	,335.80966800506377	,626.9324561102427	,1193.7167598928752	,2274.5883052693794	,2734.7612461893104	,3075.370469000327	,4235.061816130342	]
y = np.array([0.1637325958704072,0.1658702835721177,0.19384826328334373,0.2923170686360547,0.38945080711492985,0.4148250745921913,0.5015602317089521,0.6742839831054106,1.1265018636427957,1.663103233757362,1.990063198637278])/N

f_SO = interp1d(x, y, kind='linear', fill_value="extrapolate")

In [6]:
N = 1e7
x = [10,75.32337589203746	,160.85999889800618	,340.4967383339498	,871.2051805682871	,1724.809488637684	]
y = 2*np.array([1,1.0182,1.1348,1.4041,1.5198,2.5557	])/N
f_Planck = interp1d(x, y, kind='linear', fill_value="extrapolate")

In [7]:
N = 1e7
# https://arxiv.org/pdf/2403.17925
x = [35.190688373236284	,50.790359333427915	,89.97065841891472	,197.08472778011262	,385.2789162003061	,1006.4549748097186	,2181.7056856609784	,2784.791530071566	,3222.369693949373	,3725.7883975549175	,4298.874152276615	]
y = np.array([0.07856998685838304,0.08443030118214156,0.10849544565221234,0.164707182470824,0.24008135934943148,0.3648854446906422,0.5788268536804575,1.0909730047081443,1.659722689685696,1.3690446452863363,1.9974160387048998])/N
f_SPT_Summer = interp1d(x, y, kind='linear', fill_value="extrapolate")

In [8]:
N = 1e7
# https://arxiv.org/pdf/2403.17925
x = [34.150734230131775	,102.66510443231208	,256.3784422342604	,653.0266368808517	,1410.2304370377647	,2286.7839428151624	,3047.951172608016	,3301.4963574077037	,3827.9125881096406	,4746.201775079477	]
y = np.array([0.0639769792365802,0.07335548408566443,0.09262018291845851,0.12097806351013743,0.18307410239368344,0.2754037235130334,0.5975503865801056,0.7942895129566045,0.6526302418239897,1.7343096186104023])/N
f_SPT_Main = interp1d(x, y, kind='linear', fill_value="extrapolate")

In [9]:
noise_ACT = np.loadtxt('../Data/Map_Noise/ACT_noise.txt')[:,1]
noise_ACT = noise_ACT[:np.where(noise_ACT < 1e-9)[0][0]]

In [10]:
plt.figure(dpi = 120)
plt.plot(ls, ls*(ls+1)*ave_sig[ls], label = 'Signal', color = 'k', zorder=4)
plt.plot(ls_ACT, ls_ACT*(ls_ACT+1)*noise_ACT, label = 'ACT Noise', zorder=3, linestyle = '--')
plt.plot(ls, ls*(ls+1)*f_Planck(ls), label = 'Planck Noise', zorder=3, linestyle = '--')
plt.plot(ls, ls*(ls+1)*f_SO(ls), label = 'SO Noise', zorder=3, linestyle = '--')
plt.plot(ls, ls*(ls+1)*f_SPT_Main(ls), label = 'SPT Main Noise', zorder=3, linestyle = '--')
plt.plot(ls, ls*(ls+1)*f_SPT_Summer(ls), label = 'SPT Summer Noise', zorder=3, linestyle = '--')
plt.legend(fontsize=18)
# plt.grid(zorder=0)
plt.ylabel(r'$\ell (\ell +1) C_\ell$', fontsize=24)
plt.xlabel(r'$\ell$', fontsize=24)
plt.title(r'Signal and Noise of $\kappa_\text{CMB}$ Power Spectrum', fontsize=24)
plt.yscale('log')
plt.ylim([1e-5, 1e1])
plt.xlim([0, nside_ls])
# plt.show()
plt.savefig('../Paper_Plots/Signal_to_Noise.pdf', bbox_inches='tight', format='pdf')
plt.close()